In [264]:
## select nodes to map the wavelets to the brain map.

import glob
import pandas as pd
import numpy as np
import re
import os

## read harmonics:
## adni data:
input_harmonics_wavelets = 'adni_out02_harmonics'
output_dir = 'adni_out03_wavelets_brain_map'

## Notes:
# got NaN values in wavelets (e.g for ROI 3 all values are nan), 
# but time signals does not have nans...
# need to check where those NaNs are derived...

# read atlas information:
atlas_info = pd.read_csv('/home/xin/Downloads/BrainNet/Data/ExampleFiles/Power264/Node_Power264.node', 
                         header = None, 
                         delimiter = '\t')

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

harmonic_wavelets_files = glob.glob(input_harmonics_wavelets + '/CommonHarWavelets_Harmonics*.csv')
harmonic_wavelets_files.sort()

# select 6 ROIs to make brain map:
# this is not useful so far as we add arrow manually on the brain map. we don't visualize the wavelets as the node size.
harmonic_wavelets_files = harmonic_wavelets_files[10:264:50]
harmonic_wavelets_files


['adni_out02_harmonics/CommonHarWavelets_Harmonics011.csv',
 'adni_out02_harmonics/CommonHarWavelets_Harmonics061.csv',
 'adni_out02_harmonics/CommonHarWavelets_Harmonics111.csv',
 'adni_out02_harmonics/CommonHarWavelets_Harmonics161.csv',
 'adni_out02_harmonics/CommonHarWavelets_Harmonics211.csv',
 'adni_out02_harmonics/CommonHarWavelets_Harmonics261.csv']

In [269]:
## get hub nodes with eigenvector centrality:

eigen_centrality = pd.read_csv('/home/xin/Downloads/Harmonics/wd_adni/adni_out03_network_analysis/eigenvector.csv',
                               index_col = 0)

# eigen_centrality.iloc[:, 2:].mean(axis = 0)
mean_eigen_centrality = eigen_centrality.iloc[:, 2:].mean(axis = 0)
hub_nodes = eigen_centrality.iloc[:, 2:].mean(axis = 0).argsort()
print(max(hub_nodes))
hub_nodes.name = 'index'

hub_nodes = hub_nodes[-1:-41:-1]

## add harmonic wavelets values to atlas_info_hub:

for f in harmonic_wavelets_files:
    
    wavelets = pd.read_csv(f, header = None)
    roi = re.search('(.*)CommonHarWavelets_Harmonics(.*).csv', f).group(2)
    # break

    for freq in range(10):
        wavelets_freq = wavelets.iloc[hub_nodes, freq]
        wavelets_freq.index = wavelets_freq.index.astype(int)
        wavelets_freq = wavelets_freq.sort_index()

        atlas_info_hub = atlas_info.reset_index().merge(hub_nodes, on = 'index')
        atlas_info_hub[3] = wavelets_freq.values
        atlas_info_hub.iloc[:, 1:].to_csv(output_dir + '/hub_node_roi_' + str(roi) + '_freq_' + str(freq) + '.node', 
                                          sep = '\t', header = None, index = False)

# atlas_info_hub

263


In [266]:
# get the connectivity matrix for the hub nodes:
input_dir = '/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix?'
# input_files = glob.glob(input_dir + '/*.txt')
# input_files.sort()

### randomly select one subject to extract edge information:
# f = input_files[27]
f = '/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_1280-ses-2013-02-26_07_04_09.0.txt'
df_corr = pd.read_csv(f)

# print(df_corr.shape)

# use int index and columns name so that the order is correctly sorted.
df_corr.index   = [i for i in range(len(df_corr))]
df_corr.columns = [i for i in range(len(df_corr))]

## I: save connections among the hub nodes:
edge_hub = df_corr.loc[sorted(list(hub_nodes)), sorted(list(hub_nodes))].copy()

# binarize the connectivity matrix by threshold:
thresh = .6
edge_hub[edge_hub < thresh] = 0

#preserve top n connection for each node:
for col in edge_hub:
    thresh = sorted(edge_hub[col], reverse = True)[10]
    edge_hub.loc[edge_hub[col] < thresh, col] = 0

edge_hub.to_csv(output_dir + '/hub_edge.edge', sep = '\t', header = None, index = False)
    

In [276]:
## II: save neighbours for each hub node and their connections:

for i, node in enumerate(list(hub_nodes)):

    ## select top n neighbours for each hub_node:
    thresh = sorted(df_corr[node], reverse = True)[10]
    df_corr2 = df_corr.copy()
    df_corr2.loc[df_corr2[node] < thresh, node] = 0
    df_corr2.loc[node, df_corr2[node] < thresh] = 0
        
    # make node file (all neighbours of the current node):
    neighbours = df_corr2[node].to_numpy().nonzero()
    neighbours = list(neighbours[0]) 
    # print(neighbours)
    atlas_info_node = atlas_info.loc[neighbours,:]
    atlas_info_node[4] = 3
    atlas_info_node.loc[node, 4] = 5

    atlas_info_node[3] = 1
    atlas_info_node.loc[node, 3] = 2
    
    atlas_info_node.to_csv(output_dir + '/adni_hub_' + str(i) + '_node_' + str(node) + '.node', 
                           sep = '\t', header = None, index = False)
    
    ## atlas info with wavelets value:
    # unfinished...
    for f in harmonic_wavelets_files:
        
        wavelets = pd.read_csv(f, header = None)
        roi = re.search('(.*)CommonHarWavelets_Harmonics(.*).csv', f).group(2)
        # break

        for freq in range(10):
            wavelets_freq = wavelets.iloc[neighbours, freq]
            wavelets_freq.index = wavelets_freq.index.astype(int)
            wavelets_freq = wavelets_freq.sort_index()

            atlas_info_node[3] = wavelets_freq.values
            atlas_info_node.to_csv(output_dir + '/adni_hub_' + str(i) + '_node_' + str(node) + 
                                              '_wavelets_' + str(roi) + '_freq_' + str(freq) + '.node', 
                                              sep = '\t', header = None, index = False)
    
    # print(df_corr.iloc[:, node])
    # df_empty = df_corr2.loc[neighbours, neighbours].copy()
    df_corr2.loc[neighbours, neighbours][df_corr2.loc[neighbours, neighbours] < .5] = 0
    # break
    # save not to edge file:
    np.savetxt(output_dir + '/adni_edge_hub_' + str(i) + '_node_' + str(node) + '.edge', 
               np.array(df_corr2.loc[neighbours, neighbours]), 
               fmt = '%.3f',
               delimiter = '\t')

In [272]:
atlas_info_node

,0,1,2,3,4,5
0,-25,-98,-12,1,3,-
28,44,-8,57,1,3,-
96,23,33,48,1,3,-
130,-49,-42,1,1,3,-
134,11,-66,42,1,3,-
137,-10,11,67,2,5,-
195,40,18,40,1,3,-
204,42,0,47,1,3,-
206,48,22,10,1,3,-
214,0,30,27,1,3,-


In [274]:
neighbours

[0, 28, 96, 130, 134, 137, 195, 204, 206, 214, 240]